In [1]:


import os
# Find the latest version of spark 3.0  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
spark_version = 'spark-3.0.3'
os.environ['SPARK_VERSION']=spark_version
# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark
# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"
# Start a SparkSession
import findspark
findspark.init()

Hit:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:2 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:4 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:5 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:6 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Get:8 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:12 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:13 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:14 http://security.ubuntu.com/ubuntu

In [2]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2022-07-01 16:23:51--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar’

postgresql-42.2.16. 100%[===================>] 979.38K  6.14MB/s    in 0.2s    

2022-07-01 16:23:51 (6.14 MB/s) - ‘postgresql-42.2.16.jar’ saved [1002883/1002883]



In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("M16-Amazon-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

### Load Amazon Data into Spark DataFrame

In [4]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Beauty_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+---------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|          review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+---------------------+-----------+
|         US|    1797882|R3I2DHQBR577SS|B001ANOOOE|       2102612|The Naked Bee Vit...|          Beauty|          5|            0|          0|   N|                Y|          Five Stars| Love this, excell...| 2015-08-31|
|         US|   18381298|R1QNE9NQFJC2Y4|B0016J22EQ|     106393691|Alba Botanica Sun...|          Beauty|          5|

### Create DataFrames to match tables

In [5]:
from pyspark.sql.functions import to_date
# Read in the Review dataset as a DataFrame
review_id_df = df.select(["review_id", "customer_id", "product_id", "product_parent", to_date("review_date", 'yyyy-MM-dd').alias("review_date")])
review_id_df.show()

+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
|R3I2DHQBR577SS|    1797882|B001ANOOOE|       2102612| 2015-08-31|
|R1QNE9NQFJC2Y4|   18381298|B0016J22EQ|     106393691| 2015-08-31|
|R3LIDG2Q4LJBAO|   19242472|B00HU6UQAG|     375449471| 2015-08-31|
|R3KSZHPAEVPEAL|   19551372|B002HWS7RM|     255651889| 2015-08-31|
| RAI2OIG50KZ43|   14802407|B00SM99KWU|     116158747| 2015-08-31|
|R1R30FA4RB5P54|    2909389|B000NYL1Z6|     166146615| 2015-08-31|
|R30IJKCGJBGPJH|   19397215|B001SYWTFG|     111742328| 2015-08-31|
|R18GLJJPVQ1OVH|    3195210|B005F2EVMQ|     255803087| 2015-08-31|
| R8TVYIJXLYJT0|   52216383|B00M1SUW7K|     246816549| 2015-08-31|
|R1CJGF6M3PVHEZ|   10278216|B001KYQA1S|       9612905| 2015-08-31|
|R2A30ALEGLMCGN|   24655453|B00SAQ9DZY|     292127037| 2015-08-31|
|R134GP1Y3EQZ1L|   30788223|B00HFQQ0VU|     311309827| 2015-08

In [6]:
# Create the customers_table DataFrame
customers_df = df.groupby("customer_id").count().withColumnRenamed("count", "customer_count")

In [7]:
# Create the products_table DataFrame and drop duplicates. 
products_df = df.select(["product_id", "product_title"]).drop_duplicates()

In [8]:
# Create the review_id_table DataFrame. 
# Convert the 'review_date' column to a date datatype with to_date("review_date", 'yyyy-MM-dd').alias("review_date")
review_id_df = df.select(["review_id", "customer_id", "product_id", "product_parent", "review_date", to_date("review_date", 'yyyy-MM-dd').alias("review_date")])

In [9]:
# Create the vine_table. DataFrame
vine_df = df.select(["review_id", "star_rating", "helpful_votes", "total_votes", "vine", "verified_purchase"])

### Connect to the AWS RDS instance and write each DataFrame to its table. 

In [10]:
# Configure settings for RDS
mode = "append"
jdbc_url="jdbc:postgresql://challenge.cbavgt2aryin.us-east-1.rds.amazonaws.com:5432/postgres"
config = {"user":"postgres", 
          "password": "Savannah1", 
          "driver":"org.postgresql.Driver"}

In [11]:
# Write review_id_df to table in RDS
review_id_df.write.jdbc(url=jdbc_url, table='review_id_table', mode=mode, properties=config)

Py4JJavaError: ignored

In [12]:
# Write products_df to table in RDS
# about 3 min
products_df.write.jdbc(url=jdbc_url, table='products_table', mode=mode, properties=config)

Py4JJavaError: ignored

In [ ]:
# Write customers_df to table in RDS
# 5 min 14 s
customers_df.write.jdbc(url=jdbc_url, table='customers_table', mode=mode, properties=config)

In [ ]:
# Write vine_df to table in RDS
# 11 minutes
vine_df.write.jdbc(url=jdbc_url, table='vine_table', mode=mode, properties=config)

Deliverable 2

In [13]:
votes_helpful = vine_df.filter("total_votes>20").toPandas()
# votes = vine_df.toPandas()
votes_helpful

,review_id,star_rating,helpful_votes,total_votes,vine,verified_purchase
0,R2QRUE9REK8OUC,5,18,23,N,Y
1,R2POXRW6PXHXZG,5,25,30,N,Y
2,RZN43XRZ89IIJ,3,10,27,N,N
3,R29Q748WSHZ3SN,5,23,24,N,Y
4,R2UQHSNWU6WTZX,5,74,76,N,N
...,...,...,...,...,...,...
74102,R3SHWVR8GO9WK4,5,26,26,N,N
74103,R2KNC11QJB7V5U,5,30,34,N,N
74104,R2IRC0IZ8YCE5T,3,23,23,N,N
74105,R1U4ZSXOD228CZ,5,89,97,N,N


In [15]:
helpful_votes_filtered = votes_helpful.loc[(votes_helpful["helpful_votes"]/votes_helpful["total_votes"] >= 0.5)]

In [39]:
helpful_votes_filtered

,review_id,star_rating,helpful_votes,total_votes,vine,verified_purchase
0,R2QRUE9REK8OUC,5,18,23,N,Y
1,R2POXRW6PXHXZG,5,25,30,N,Y
3,R29Q748WSHZ3SN,5,23,24,N,Y
4,R2UQHSNWU6WTZX,5,74,76,N,N
5,R2HSRI3D6E2M9Y,5,69,71,N,Y
...,...,...,...,...,...,...
74102,R3SHWVR8GO9WK4,5,26,26,N,N
74103,R2KNC11QJB7V5U,5,30,34,N,N
74104,R2IRC0IZ8YCE5T,3,23,23,N,N
74105,R1U4ZSXOD228CZ,5,89,97,N,N


In [52]:
vine_paid=helpful_votes_filtered.query("vine == 'Y'")

In [53]:
vine_paid

,review_id,star_rating,helpful_votes,total_votes,vine,verified_purchase
243,R106V6GUNWRSSS,5,857,889,Y,N
275,R37N8KZS48W36A,5,36,37,Y,N
421,R1LNEQ2MQE03PD,5,49,66,Y,N
449,R1ECPWPUNK36ES,5,29,31,Y,N
700,R16BUM7UQZLOM6,3,107,112,Y,N
...,...,...,...,...,...,...
70080,R23P18F8OSXQXL,5,38,40,Y,N
70112,R39VHZ5XXVYKN,5,22,27,Y,N
70113,R2F4KBBD3MTMKP,5,390,423,Y,N
70122,RQY6RA1SE9R3H,5,20,23,Y,N


In [54]:
vine_unpaid=helpful_votes_filtered.query("vine == 'N'")

In [55]:
vine_unpaid

,review_id,star_rating,helpful_votes,total_votes,vine,verified_purchase
0,R2QRUE9REK8OUC,5,18,23,N,Y
1,R2POXRW6PXHXZG,5,25,30,N,Y
3,R29Q748WSHZ3SN,5,23,24,N,Y
4,R2UQHSNWU6WTZX,5,74,76,N,N
5,R2HSRI3D6E2M9Y,5,69,71,N,Y
...,...,...,...,...,...,...
74102,R3SHWVR8GO9WK4,5,26,26,N,N
74103,R2KNC11QJB7V5U,5,30,34,N,N
74104,R2IRC0IZ8YCE5T,3,23,23,N,N
74105,R1U4ZSXOD228CZ,5,89,97,N,N


In [56]:
vine_paid_star = vine_paid.query("star_rating =='5'")
vine_paid_star

,review_id,star_rating,helpful_votes,total_votes,vine,verified_purchase
243,R106V6GUNWRSSS,5,857,889,Y,N
275,R37N8KZS48W36A,5,36,37,Y,N
421,R1LNEQ2MQE03PD,5,49,66,Y,N
449,R1ECPWPUNK36ES,5,29,31,Y,N
904,R7NWIOCA5RVPR,5,19,26,Y,N
...,...,...,...,...,...,...
70080,R23P18F8OSXQXL,5,38,40,Y,N
70112,R39VHZ5XXVYKN,5,22,27,Y,N
70113,R2F4KBBD3MTMKP,5,390,423,Y,N
70122,RQY6RA1SE9R3H,5,20,23,Y,N


In [57]:
vine_unpaid_star = vine_unpaid.query("star_rating =='5'")
vine_unpaid_star

,review_id,star_rating,helpful_votes,total_votes,vine,verified_purchase
0,R2QRUE9REK8OUC,5,18,23,N,Y
1,R2POXRW6PXHXZG,5,25,30,N,Y
3,R29Q748WSHZ3SN,5,23,24,N,Y
4,R2UQHSNWU6WTZX,5,74,76,N,N
5,R2HSRI3D6E2M9Y,5,69,71,N,Y
...,...,...,...,...,...,...
74101,RQLSGNI4JL4O7,5,36,37,N,N
74102,R3SHWVR8GO9WK4,5,26,26,N,N
74103,R2KNC11QJB7V5U,5,30,34,N,N
74105,R1U4ZSXOD228CZ,5,89,97,N,N
